# Análisis de Ventas con SQL + Pandas

In [1]:
import pandas as pd
import duckdb
import matplotlib.pyplot as plt
import os
import zipfile
import requests

os.makedirs("../data",exist_ok=True)
os.makedirs("../results",exist_ok=True)

In [2]:
# Descargar los datos
url = "https://archive.ics.uci.edu/static/public/502/online+retail+ii.zip"
zip_path = "../data/online_retail_II.zip"

# Descargar el archivo
response = requests.get(url)
with open(zip_path, 'wb') as f:
    f.write(response.content)
print(f"Archivo descargado: {zip_path}")

# Extraer el contenido del .zip
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall("../data")
    extracted_file = os.path.join("../data", "online_retail_II.xlsx")
print(f"Archivo extraído: {extracted_file}")

# Eliminar el .zip original
os.remove(zip_path)
print(f"Archivo .zip eliminado: {zip_path}")

# Cargar el archivo de Excel
archivo_excel = pd.ExcelFile(extracted_file)

# Ver el número de hojas
numero_hojas = len(archivo_excel.sheet_names)
print(f"El archivo tiene {numero_hojas} hojas.")

# Ver los nombres de las hojas
nombres_hojas = archivo_excel.sheet_names
print("Nombres de las hojas:", nombres_hojas)

# Leer la segunda hoja (índice 1) y guardar como CSV
df = pd.read_excel(extracted_file, sheet_name=1)
file_path = "../data/online_retail_II.csv"
df.to_csv(file_path, index=False)
print(f"Archivo CSV guardado en: {file_path}")

Archivo descargado: ../data/online_retail_II.zip
Archivo extraído: ../data/online_retail_II.xlsx
Archivo .zip eliminado: ../data/online_retail_II.zip
El archivo tiene 2 hojas.
Nombres de las hojas: ['Year 2009-2010', 'Year 2010-2011']
Archivo CSV guardado en: ../data/online_retail_II.csv


In [3]:
db=duckdb.read_csv(file_path)

In [4]:
# Leer el archivo CSV y asignarlo a una tabla temporal llamada 'db'
query = """
    SELECT *
    FROM db
    WHERE Country = 'United Kingdom'
    LIMIT 10;
"""

# Ejecutar la consulta
resultado = duckdb.sql(query)

# Mostrar el resultado
resultado.df()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
5,536365,22752,SET 7 BABUSHKA NESTING BOXES,2,2010-12-01 08:26:00,7.65,17850.0,United Kingdom
6,536365,21730,GLASS STAR FROSTED T-LIGHT HOLDER,6,2010-12-01 08:26:00,4.25,17850.0,United Kingdom
7,536366,22633,HAND WARMER UNION JACK,6,2010-12-01 08:28:00,1.85,17850.0,United Kingdom
8,536366,22632,HAND WARMER RED POLKA DOT,6,2010-12-01 08:28:00,1.85,17850.0,United Kingdom
9,536368,22960,JAM MAKING SET WITH JARS,6,2010-12-01 08:34:00,4.25,13047.0,United Kingdom
